In [1]:
### This script trains a neural network to predict whether a binary string is divisible by 7

from __future__ import print_function
import numpy as np, re
from scipy import stats
import sklearn.metrics, math
from math import floor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from scipy.stats import spearmanr, pearsonr
from matplotlib.colors import ListedColormap
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pickle
from math import pow

In [2]:
# Disable tensorflow warnings
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import add
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import logcosh
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras import losses
from time import time
import tensorflow.keras as keras

from random import randint
import random
import sys
from random import randrange

from tensorflow.keras import backend as K
import keract

In [6]:




def clean_up_models(prefix, max_epoch_num):
    '''Deletes suboptimal models and removes epoch number from the best model'''
    for c in np.arange(1, max_epoch_num+1):
        try:
            os.rename(prefix+'-'+'{:02d}'.format(c)+'.hdf5', prefix+'.hdf5')
        except:
            continue
    return 0


def lr_scheduler(epoch):
    if epoch < 5:
        return 2e-3
    elif epoch < 45:
        return 1e-3
    else:
        return 5e-4


def create_nn(N = 1, num_filters = 8, conv_kernel_size = 5, num_convolutions = 1):

    inputShape = (4, 10) #(N, 1)
    chanDim = -1

    # define the model input
    inputs = Input(shape=inputShape)
    x = inputs
    # x = BatchNormalization(axis=chanDim)(x)

    # # Perform multiple convolutional layers
    # for i in range(num_convolutions):
    #     x = Conv1D(kernel_size=conv_kernel_size, filters=num_filters, padding='same')(x)
    #     x = BatchNormalization(axis=chanDim)(x)
    #     x = Activation('relu')(x)
    #     x = MaxPooling1D(pool_size=2)(x)

    x = Flatten()(x)

    x = Dense(4, activation='relu')(x) 
    # x = BatchNormalization(axis=chanDim)(x) 

    x = Dense(2)(x) 
    # x = BatchNormalization(axis=chanDim)(x) 

    x = Dense(1, activation='linear')(x)

    # construct the NN
    model = Model(inputs, x)
    return model


def to_binary(input, N):
    binary_function = lambda x : ''.join(reversed( [str((x >> i) & 1) for i in range(N)] ) )
    binary = binary_function(input)
    l = np.asarray([int(x) for x in list(binary)])
    return np.expand_dims(l, 1)


def sum_pairs(i, j, p, q):
    return int(''.join([str(i), str(j)])) + int(''.join([str(p), str(q)]))


def one_hot_encode_integer(n):
    x = np.zeros((1, 10))
    x[0][n] = 1
    return x

def create_one_hot_encoded_addition(i, j, p, q):
    x = np.concatenate((one_hot_encode_integer(i), one_hot_encode_integer(j), one_hot_encode_integer(p), one_hot_encode_integer(q)), axis = 0)
    print(x.shape)
    return x

def reverse_one_hot_encoding_of_integer(input):
    # shape of input is (4, 10)
    i = list(input[0,:]).index(1)
    j = list(input[1,:]).index(1)

    p = list(input[2,:]).index(1)
    q = list(input[3,:]).index(1)

    return [i, j, p, q]


def xi_cor(x, y):
    # First break ties by shuffling both lists with the same permutation
    z = list(zip(x, y))
    random.shuffle(z)
    X, Y = zip(*z)

    # Now, sort the two lists based on the first list
    y_sorted = list([b for a, b in sorted(zip(Y, X))])

    # Compute the ranks of the elements of y
    y_ri = [sum(u <= v for u in y_sorted) for v in y_sorted] 
    y_li = [sum(u >= v for u in y_sorted) for v in y_sorted] 

    # Compute the xi correlation
    n = len(y)

    numerator = 0
    for i in range(0,n-1):
        numerator += abs(y_ri[i+1] - y_ri[i])

    denominator = 0
    for i in range(0, n):
        denominator += y_li[i] * (n - y_li[i])

    xi = 1 - ( ((n * 1.0) * numerator) / (2.0 * denominator) )

    return xi

if __name__ == '__main__':

    N = 14 # int(sys.argv[1])
    run_name = str(sys.argv[1]) + "_" + str(N)

    output_file = run_name+".txt"

    # ######################################################################################################
    # # Create dataset for classification
    # positive_dataset = []
    # negative_dataset = []

    # ###################################################
    # # Divisibility by seven
    # for i in range(1, int(pow(2,N))):
    #     if(i%5==0):
    #         positive_dataset.append(to_binary(i, N))
    #     else:
    #         negative_dataset.append(to_binary(i, N))
    # positive_dataset_size = int(pow(2, N-3))
    # negative_dataset_size = int(pow(2, N-1))
    # ###################################################

    # ######################################################################################################

    ###################################################
    # Sum two one-hot encoded numbers
    X = []
    Y = []
    for i in range(int(pow(10,4))):
        i = randint(0, 9)
        j = randint(0, 9)             

        p = randint(0, 9)
        q = randint(0, 9)

        sum_val = sum_pairs(i, j, p, q)
        X.append(create_one_hot_encoded_addition(i, j, p, q))
        Y.append(sum_val)

    X = np.asarray(X)
    Y = np.asarray(Y)

    xIndices = list(np.random.choice(range(X.shape[0]), np.shape(X)[0], replace = False))
    xTrainIndices =  xIndices[0:int(0.6*X.shape[0])-1]
    xValidationIndices = xIndices[int(0.6*X.shape[0])+1:int(0.8*X.shape[0])-1]
    xTestIndices = xIndices[int(0.9*X.shape[0])+1:int(1.0*X.shape[0])-1]

    xTrain = [X[i] for i in xTrainIndices]
    xValidation = [X[i] for i in xValidationIndices]
    xTest = [X[i] for i in xTestIndices]

    xTrain = np.asarray(xTrain)
    xValidation = np.asarray(xValidation)
    xTest = np.asarray(xTest)

    yTrain = [Y[i] for i in xTrainIndices]
    yValidation = [Y[i] for i in xValidationIndices]
    yTest = [Y[i] for i in xTestIndices]

    yTrain = np.squeeze(np.asarray(yTrain))
    yValidation = np.squeeze(np.asarray(yValidation))
    yTest = np.squeeze(np.asarray(yTest))

    np.save(run_name+".xTrain", xTrain)
    np.save(run_name+".yTrain", yTrain)

    np.save(run_name+".xValidation", xValidation)
    np.save(run_name+".yValidation", yValidation)

    np.save(run_name+".xTest", xTest)
    np.save(run_name+".yTest", yTest)

    print(xTrain[0].shape, "\n", xTrain[0])
    print(xTrain.shape, xValidation.shape, xTest.shape)
    print(yTrain.shape, yValidation.shape, yTest.shape)

    # Now we train a NN    
    num_epochs = 50
    # steps_per_epoch = 100
    # validation_steps_per_epoch = 100 # keeping this high is good because it chooses the truly best model
    batch_size = 32 # Increase batch size significantly 
    num_filters = 32 # Use smaller values # Increasing this matters the most: Peaks around 128?
    conv_kernel_size = 5 # Maybe keep this at 11? 
    num_convolutions = 3
    learning_rate = 1e-3 
    filepath = run_name+"model-{epoch:02d}.hdf5"
    checkpoint = ModelCheckpoint(filepath, verbose=0, save_best_only=False)
    lr_schedule = LearningRateScheduler(lr_scheduler)
    # tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq = 1, embeddings_freq = 0, write_graph = True, write_images = True, write_grads = 1, update_freq = 'epoch')
    # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1)
    # earlystopping=EarlyStopping(monitor="mean_squared_error", patience=40, verbose=1, mode='auto')

    # These metrics are not being used because TODO: how to load them with the model
    METRICS = [
                keras.metrics.TruePositives(name='tp'),
                keras.metrics.FalsePositives(name='fp'),
                keras.metrics.TrueNegatives(name='tn'),
                keras.metrics.FalseNegatives(name='fn'), 
                keras.metrics.BinaryAccuracy(name='accuracy'),
                keras.metrics.Precision(name='precision'),
                keras.metrics.Recall(name='recall'),
                keras.metrics.AUC(name='auroc', curve='ROC'),
                keras.metrics.AUC(name='auprc', curve='PR')
    ]

    # # Initialize bias 
    # pos_train = list(yTrain).count(1)
    # neg_train = list(yTrain).count(0)
    # total_train = pos_train + neg_train
    # print(pos_train, neg_train, total_train)
    # initial_bias = np.log([pos_train/neg_train]) # ERROR

    # # Set class weights -> is this compatible with focal loss?
    # weight_for_1 = (1 / pos_train)*(total_train)/2.0
    # weight_for_0 = (1 / neg_train)*(total_train)/2.0 
    # class_weight = {1: weight_for_1, 0: weight_for_0}

    callbacks_list = [checkpoint, lr_schedule] #, tensorboard]#, reduce_lr]#, earlystopping]
    model = create_nn(N, num_filters, conv_kernel_size, num_convolutions)
    opt = Adam(lr=learning_rate)
    model.compile(loss='mean_squared_error', #losses.binary_crossentropy, # loss = [focal_loss]
    			 optimizer=opt, metrics=['mse'])# [METRICS])
    print(model.summary())
    # plot_model(model, to_file=run_name+'.model.png', show_shapes=True)
    print("Ready to train")
    result = model.fit(xTrain, yTrain,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        shuffle=True,
                        verbose=1,
                        # class_weight=class_weight,
                        validation_data= (xValidation, yValidation),
                        callbacks=callbacks_list)


    # Interpret the model
    activations = keract.get_activations(model, xTest, auto_compile=True)

    [print(k, '->', v.shape, '- Numpy array') for (k, v) in activations.items()]
    
    xTest_reversed = np.asarray([reverse_one_hot_encoding_of_integer(x) for x in xTest])
    print(xTest_reversed.shape, xTest_reversed[127], yTest[127])

    xi_d = {}
    for i in range(xTest_reversed.shape[1]):
        x = xTest_reversed[:,i]

        for j in range(activations['dense'].shape[1]):
            y = np.asarray(activations['dense'][:,j])
            xi_d[(i, j, 0)] = xi_cor(list(x), list(y))
            print((i, j, 0), xi_d[(i, j, 0)])

        for k in range(activations['dense_1'].shape[1]):
            y = np.asarray(activations['dense_1'][:,k])
            xi_d[(i, 0, k)] = xi_cor(list(x), list(y))
            print((i, 0, k), xi_d[(i, 0, k)])

    for j in range(activations['dense'].shape[1]):
        x = np.asarray(activations['dense'][:,j])
        for k in range(activations['dense_1'].shape[1]):
            y = np.asarray(activations['dense_1'][:,k])       
            xi_d[(-1, j, k)] = xi_cor(list(x), list(y))
            print((-1, j, k), xi_d[(-1, j, k)])

    for key in xi_d:
        print(key, xi_d[key])




    # for layer in model.layers:
    #     print(layer, layer.output_shape)
    # func = K.function([model.layers[0].input], [model.layers[2].output])
    # print( np.asarray(xTest[11]).shape, np.asarray(func([xTest[11]])).shape )


    # # Output the 
    # inp = model.input                                           
    # outputs = [layer.output for layer in model.layers]          
    # functors = [K.function([inp], [out]) for out in outputs] #K.function([inp, K.learning_phase()], outputs) 

    # # input_shape = (4, 10)
    # # test = np.random.random(input_shape)[np.newaxis,...]
    # layer_outs = [func([xTest[0]]) for func in functors]
    # print(layer_outs)

    # sys.exit(1)

    # # Not doing this because we will evaluate all models using top-k accuracy
    # Load best model
    clean_up_models(run_name+'model', num_epochs)
    final_model = load_model(run_name+'model.hdf5')
    os.rename(run_name+'model.hdf5', run_name+'.model.hdf5')

    # # Now evaluate metrics on all trained models for each epoch
    # xTrain = np.load(run_name+".xTrain"+".npy")
    # yTrain = np.load(run_name+".yTrain"+".npy")

    # xValidation = np.load(run_name+".xValidation"+".npy")
    # yValidation = np.load(run_name+".yValidation"+".npy")

    # xTest = np.load(run_name+".xTest"+".npy")
    # yTest = np.load(run_name+".yTest"+".npy")

    # Validation_top_k_epoch = {}
    # for model_index in range(1, num_epochs+1):
    #     final_model = load_model(run_name+'model-'+'{:02d}'.format(model_index)+'.hdf5')
        
    #     # Predict on Validation data in batches
    #     yValidation_Pred = []
    #     yValidation_Pred = final_model.predict(xValidation, batch_size = 256, verbose = 1)
    #     yValidation_Pred = np.squeeze(yValidation_Pred)

    #     size_yValidation = list(yValidation).count(1)
    #     top_K_accuracy_Validation = []
    #     for K in list( range(size_yValidation, size_yValidation+1)   ):    
    #         top_K_accuracy_Validation.append( round( 100.0 * ( (list(yValidation[np.asarray(yValidation_Pred).argsort()[(-1*K):][::-1]]).count(1)*1.0) / (1.0*K)   ),  3) )                
    #     print("Model", model_index, "Top K accuracy Validation =", top_K_accuracy_Validation[0], file=open(output_file, "a"))
    #     Validation_top_k_epoch[model_index] = top_K_accuracy_Validation[0]

    # # Once the model with the highest validation Top K accuracy has been chosen, 
    # # compute metrics on the test data and evaluate 
    # max_top_k = -1
    # max_top_k_index = -1
    # for model_index in range(1, num_epochs+1):
    #     if(Validation_top_k_epoch[model_index] > max_top_k):
    #         max_top_k_index = model_index
    #         max_top_k = Validation_top_k_epoch[model_index]

    # os.rename(run_name+'model-'+'{:02d}'.format(max_top_k_index)+'.hdf5', run_name+'.model.hdf5')
    # final_model = load_model(run_name+'.model.hdf5')
    # clean_up_models(run_name+'model', num_epochs)
    # os.remove(run_name+'model.hdf5')

    yTrain_Pred = []
    yTrain_Pred = final_model.predict(xTrain, batch_size = 256, verbose = 1)
    yTrain_Pred = np.squeeze(yTrain_Pred)

    yTest_Pred = []
    yTest_Pred = final_model.predict(xTest, batch_size = 256, verbose = 1)
    yTest_Pred = np.squeeze(yTest_Pred)

    print(yTest, yTest_Pred)
    sys.exit(1)

    # # Compute AUROC for Train and Test data
    roc_auc_train = round(roc_auc_score(yTrain, yTrain_Pred), 3)
    print("Model", max_top_k_index, "AUROC Train = "+str(roc_auc_train), file=open(output_file, "a"))
    roc_auc_test = round(roc_auc_score(yTest, yTest_Pred), 3)
    print("Model", max_top_k_index, "AUROC Test = "+str(roc_auc_test), file=open(output_file, "a"))

    # # Compute ROC Curve for Train and Test data
    fpr_train, tpr_train, _ = roc_curve(yTrain, yTrain_Pred)
    fpr_test, tpr_test, _ = roc_curve(yTest, yTest_Pred)
    plt.plot(fpr_train, tpr_train, color='blue', label="Train")
    plt.plot(fpr_test, tpr_test, color='green', label="Test")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='upper left')
    plt.title('ROC AUC Train = '+str(round(roc_auc_train,3))+"%"+" ROC AUC Test = "+str(round(roc_auc_test,3))+"%")
    axes = plt.gca()
    axes.set_ylim([-0.2,1.2])
    plt.savefig(run_name+".Model_"+str(max_top_k_index)+'.ROC_AUC.png')
    plt.close()

    # Compute AUPRC for Train and Test data
    precision_train, recall_train, _ = precision_recall_curve(yTrain, yTrain_Pred)
    auprc_train = round(auc(recall_train, precision_train), 3)
    print("Model", max_top_k_index, "AUPRC Train = "+str(auprc_train), file=open(output_file, "a"))
    precision_test, recall_test, _ = precision_recall_curve(yTest, yTest_Pred)
    auprc_test = round(auc(recall_test, precision_test), 3)
    print("Model", max_top_k_index, "AUPRC Test = "+str(auprc_test), file=open(output_file, "a"))

    # Compute PR Curve for Train and Test data
    plt.plot(recall_train, precision_train, color='blue', label="Train")
    plt.plot(recall_test, precision_test, color='green', label="Test")
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend(loc='upper left')
    plt.title("AUPRC Train = "+str(round(auprc_train,3))+"%"+" AUPRC Test = "+str(round(auprc_test,3))+"%")
    axes = plt.gca()
    axes.set_ylim([-0.2,1.2])
    plt.savefig(run_name+'.Model_'+str(max_top_k_index)+'.AUPRC.png')
    plt.close()


    size_yTrain = list(yTrain).count(1)
    top_K_accuracy_Train = []
    for K in list( range(size_yTrain, size_yTrain+1)   ):    
        top_K_accuracy_Train.append( round( 100.0 * ( (list(yTrain[np.asarray(yTrain_Pred).argsort()[(-1*K):][::-1]]).count(1)*1.0) / (1.0*K)   ),  3) )
    print("Model", max_top_k_index, "Top K accuracy Train =", top_K_accuracy_Train[0], file=open(output_file, "a"))

    size_yTest = list(yTest).count(1)
    top_K_accuracy_Test = []
    for K in list( range(size_yTest, size_yTest+1)   ):    
        top_K_accuracy_Test.append( round( 100.0 * ( (list(yTest[np.asarray(yTest_Pred).argsort()[(-1*K):][::-1]]).count(1)*1.0) / (1.0*K)   ),  3) )
    print("Model", max_top_k_index, "Top K accuracy Test =", top_K_accuracy_Test[0], file=open(output_file, "a"))

    # At this point delete the xTrain, yTrain, xValidation, yValidation preserving xTest, yTest
    os.remove(run_name+".xTrain"+".npy")
    os.remove(run_name+".yTrain"+".npy")
    os.remove(run_name+".xValidation"+".npy")
    os.remove(run_name+".yValidation"+".npy")
    os.remove(run_name+".xTest"+".npy")
    os.remove(run_name+".yTest"+".npy")

    print("Finished analyzing predictions on the Train, Validation and Test data")




(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)


(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)


(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)


(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)


(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)
(4, 10)


...

(998, 4) [7 2 6 1] 133
(0, 0, 0) 0.32300990571849364
(0, 1, 0) 0.41821306531739544
(0, 2, 0) 0.30840965149209776
(0, 3, 0) 0.9972590261151948
(0, 0, 0) 0.346058450563167
(0, 0, 1) 0.3619926454135014
(1, 0, 0) 0.06651379570162452
(1, 1, 0) 0.036416685737641896
(1, 2, 0) 0.039024410607849114
(1, 3, 0) 0.9972214767757207
(1, 0, 0) 0.05213013431227076
(1, 0, 1) 0.04700313371950082
(2, 0, 0) 0.3569840418958259
(2, 1, 0) 0.3069742158069375
(2, 2, 0) 0.3906028193296486
(2, 3, 0) 0.9972504751007685
(2, 0, 0) 0.353437245719309
(2, 0, 1) 0.3560018521854058
(3, 0, 0) 0.06252209282479959
(3, 1, 0) 0.05720402788166501
(3, 2, 0) 0.056766717193414196
(3, 3, 0) 0.9972568693191537
(3, 0, 0) 0.0655282215419366
(3, 0, 1) 0.05380034399339084
(-1, 0, 0) 0.8534975230374123
(-1, 0, 1) 0.8040418675982757
(-1, 1, 0) 0.8275655560932892
(-1, 1, 1) 0.8432797506163439
(-1, 2, 0) 0.8134916421286799
(-1, 2, 1) 0.8529047859068811


C:\Users\aidan\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:112: RuntimeWarning: invalid value encountered in double_scalars


(-1, 3, 0) nan
(-1, 3, 1) nan
(0, 0, 0) 0.346058450563167
(0, 1, 0) 0.41821306531739544
(0, 2, 0) 0.30840965149209776
(0, 3, 0) 0.9972590261151948
(0, 0, 1) 0.3619926454135014
(1, 0, 0) 0.05213013431227076
(1, 1, 0) 0.036416685737641896
(1, 2, 0) 0.039024410607849114
(1, 3, 0) 0.9972214767757207
(1, 0, 1) 0.04700313371950082
(2, 0, 0) 0.353437245719309
(2, 1, 0) 0.3069742158069375
(2, 2, 0) 0.3906028193296486
(2, 3, 0) 0.9972504751007685
(2, 0, 1) 0.3560018521854058
(3, 0, 0) 0.0655282215419366
(3, 1, 0) 0.05720402788166501
(3, 2, 0) 0.056766717193414196
(3, 3, 0) 0.9972568693191537
(3, 0, 1) 0.05380034399339084
(-1, 0, 0) 0.8534975230374123
(-1, 0, 1) 0.8040418675982757
(-1, 1, 0) 0.8275655560932892
(-1, 1, 1) 0.8432797506163439
(-1, 2, 0) 0.8134916421286799
(-1, 2, 1) 0.8529047859068811
(-1, 3, 0) nan
(-1, 3, 1) nan
998/998 [==============================] - 0s 6us/sample
[ 82  84  94  55 113  93  83  68 188 172  99  89  83  37 140 135  68  82
 107  95  61  88 143 142 123 147  98 131

SystemExit: 1

C:\Users\aidan\anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
